In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 데이터 로드
data = pd.read_csv('abalone.csv')

# ID 컬럼 드롭: ID 컬럼은 모델 학습에 필요하지 않으므로 제거합니다.
data = data.drop('id', axis=1)

# 원핫 인코딩: 'Sex'와 같은 범주형 변수를 원핫 인코딩합니다.
data = pd.get_dummies(data, columns=['Sex'], drop_first=True)  # drop_first=True로 첫 번째 카테고리를 제거하여 다중공선성 방지

#다중공선성이란? 다중공선성은 회귀 분석이나 다른 통계 모델에서 독립 변수들 간에 강한 상관관계가 있을 때 발생합니다.
#이는 모델의 해석을 어렵게 만들고, 회귀 계수의 신뢰성을 떨어뜨릴 수 있습니다.
#예를 들어, 'Sex'라는 변수에 'Male', 'Female', 'I'라는 세 가지 카테고리가 있다면, 원핫 인코딩 후 다음과 같은 변수가 생성됩니다:Sex_Male, Sex_Female, Sex_I
#예를 들어, 만약 Sex_Male이 1이면 나머지 두 변수는 0이 됩니다. 이러한 중복 정보가 모델에 포함되면, 다중공선성이 발생합니다.
#drop_first=True를 설정하면 첫 번째 카테고리(여기서는 Sex_Male)가 제거됩니다.
#따라서, 남은 변수는 Sex_Female와 Sex_I만이 됩니다. 이를 통해 모델은 필요한 정보는 그대로 유지하면서, 다중공선성을 줄일 수 있습니다.


# 특성과 레이블 분리: 'Rings' 컬럼을 타겟으로 설정합니다.
X = data.drop('Rings', axis=1)  # 'Rings' 컬럼을 제외한 입력 데이터
y = data['Rings']  # 타겟 변수

# 데이터 분할: 전체 데이터를 학습 세트와 테스트 세트로 나눕니다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 스케일링: 특성 값을 정규화하여 모델 학습의 안정성을 높입니다.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)  # 학습 데이터에 맞춰 스케일링
X_test = scaler.transform(X_test)  # 테스트 데이터는 학습 데이터의 스케일로 변환


In [4]:
import numpy as np

# 2D 형태로 변환: CNN은 2D 입력을 요구하므로 데이터를 재구성합니다.
height = 1  # 높이 : 각 샘플이 1행으로 구
width = X_train.shape[1]  # 너비 = 특성 수 = X_train의 열 수
channels = 1  # 채널 수 (흑백 이미지) <-여기에서는 단일 채널을 의미합니다.

X_train_cnn = X_train.reshape((X_train.shape[0], height, width, channels))
X_test_cnn = X_test.reshape((X_test.shape[0], height, width, channels))


In [6]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


# CNN 모델 구축

cnn_model = Sequential()  # 순차 모델 생성 : 층을 순서대로 쌓으려고
cnn_model.add(Conv2D(32, (1, 3), activation='relu', input_shape=(height, width, channels)))  # 합성곱 층 추가
# Conv2D: 32개의 필터를 사용하여 (1, 3) 크기의 커널로 합성곱 수행
# relu 활성화 함수를 사용하여 비선형성을 추가

cnn_model.add(MaxPooling2D(pool_size=(1, 2)))  # 최대 풀링 층 추가
# MaxPooling2D: (1, 2) 크기의 풀링을 적용하여 특성 맵의 크기를 줄임

cnn_model.add(Flatten())  # 평탄화 층 추가
# Flatten: 다차원 배열을 1차원 배열로 변환하여 밀집층에 연결합니다.

cnn_model.add(Dense(64, activation='relu'))  # 밀집층 추가
# Dense: 64개의 뉴런을 가진 밀집층을 추가합니다.

cnn_model.add(Dense(1))  # 출력 층 추가
# 출력 층: 회귀 문제이므로 출력 뉴런 수를 1로 설정합니다.

C:\Users\tyra\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
#  모델 컴파일 및 학습
cnn_model.compile(loss='mean_squared_error', optimizer='adam')
cnn_model.fit(X_train_cnn, y_train, epochs=50, batch_size=32)

Epoch 1/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 79.2358
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 17.9101
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8.5609
Epoch 4/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8.2835
Epoch 5/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 6.9586
Epoch 6/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.8790
Epoch 7/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.9691
Epoch 8/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.5844
Epoch 9/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.1572
Epoch 10/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4.6537
Epoch 11/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4.5890
Epoch 12/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4.5716
Epoch 13/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4.7906
Epoch 14/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.0592
Epoch 15/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - 

In [8]:
# 모델 평가 및 예측
cnn_score = cnn_model.evaluate(X_test_cnn, y_test)
print(f'Test Loss: {cnn_score}')

cnn_predictions = cnn_model.predict(X_test_cnn)

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 843us/step - loss: 4.8582
Test Loss: 4.633538246154785
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


### 예측 손실이 4.63....이다. 레이어 늘리고 드롭아웃도 넣어보자

In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# 1. 데이터 로드
data = pd.read_csv('abalone.csv')
data = data.drop('id', axis=1)

# 2. 원핫 인코딩
data = pd.get_dummies(data, columns=['Sex'], drop_first=True)

# 3. 특성과 레이블 분리
X = data.drop('Rings', axis=1)
y = data['Rings']

# 4. 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 5. 스케일링
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 6. 데이터 형태 변환
n_samples = X_train.shape[0]
n_features = X_train.shape[1]

# 높이와 너비 자동 설정
height = 2 if n_features % 2 == 0 else 1
width = n_features // height
channels = 1

# 데이터 형태를 (샘플 수, 높이, 너비, 채널 수)로 재구성
X_train_cnn = X_train.reshape((n_samples, height, width, channels))
X_test_cnn = X_test.reshape((X_test.shape[0], height, width, channels))

# 7. CNN 모델 구축
cnn_model = Sequential()
cnn_model.add(Conv2D(32, (1, 2), activation='relu', input_shape=(height, width, channels)))  # 커널 크기를 (1, 2)로 설정
cnn_model.add(MaxPooling2D(pool_size=(1, 2)))  # 풀링 크기를 (1, 2)로 설정
cnn_model.add(Dropout(0.3))  # 드롭아웃 추가

cnn_model.add(Conv2D(64, (1, 2), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(1, 2)))  # 풀링 크기를 (1, 2)로 설정

cnn_model.add(Flatten())
cnn_model.add(Dense(64, activation='relu'))
cnn_model.add(Dense(1))

# 8. 모델 컴파일 및 학습
cnn_model.compile(loss='mean_squared_error', optimizer='adam')
cnn_model.fit(X_train_cnn, y_train, epochs=100, batch_size=32)

# 9. 모델 평가 및 예측
cnn_score = cnn_model.evaluate(X_test_cnn, y_test)
print(f'Test Loss: {cnn_score}')

cnn_predictions = cnn_model.predict(X_test_cnn)


Epoch 1/100


C:\Users\tyra\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


105/105 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 72.1235
Epoch 2/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 15.0259
Epoch 3/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10.0803
Epoch 4/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.4132
Epoch 5/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10.4222
Epoch 6/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.8435
Epoch 7/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6.8228
Epoch 8/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6.9784
Epoch 9/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.3295
Epoch 10/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6.9346
Epoch 11/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.4021
Epoch 12/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.0038
Epoch 13/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6.5915
Epoch 14/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6.7066
Epoch 15/100
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/st

### 더 안 좋아졌다... 아마도 overfitting이 된 거 같으니 케라스의 Earlystopping을 사용해 과적합을 막아보도록 하자.

In [31]:
from keras.callbacks import EarlyStopping

# CNN 모델 구축
cnn_model = Sequential()
cnn_model.add(Conv2D(32, (1, 2), activation='relu', input_shape=(height, width, channels)))
cnn_model.add(MaxPooling2D(pool_size=(1, 2)))
cnn_model.add(Dropout(0.5))

cnn_model.add(Conv2D(64, (1, 2), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(1, 2)))
cnn_model.add(Dropout(0.5))

cnn_model.add(Flatten())
cnn_model.add(Dense(64, activation='relu'))
cnn_model.add(Dense(1))

# 모델 컴파일
cnn_model.compile(loss='mean_squared_error', optimizer='adam')

# EarlyStopping 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 모델 학습 (검증 데이터도 제공)
cnn_model.fit(X_train_cnn, y_train, epochs=200, batch_size=32, 
              validation_split=0.2,  # 훈련 데이터의 20%를 검증 데이터로 사용
              callbacks=[early_stopping])  # EarlyStopping 콜백 추가

# 모델 평가
cnn_score = cnn_model.evaluate(X_test_cnn, y_test)
print(f'Test Loss: {cnn_score}')


Epoch 1/200
84/84 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 78.8725 - val_loss: 19.2109
Epoch 2/200
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 18.9712 - val_loss: 11.0049
Epoch 3/200
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 20.5676 - val_loss: 8.7142
Epoch 4/200
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 11.8114 - val_loss: 8.5603
Epoch 5/200
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 12.3335 - val_loss: 7.8889
Epoch 6/200
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 10.1658 - val_loss: 7.4375
Epoch 7/200
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 11.7338 - val_loss: 8.0377
Epoch 8/200
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.7267 - val_loss: 7.2041
Epoch 9/200
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10.1362 - val_loss: 6.9968
Epoch 10/200
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.5303 - val_loss: 7.6697
Epoch 11/200
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.8255 - val_loss: 7.7619
Epoch 12/200
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step -

### 내가 만질수록 모델이 안 좋아지고 있다. 필터도 줄이고 뉴런도 줄이고 학습률도 줄여보자

In [33]:
from keras.callbacks import EarlyStopping

# CNN 모델 구축 (간소화)
cnn_model = Sequential()
cnn_model.add(Conv2D(16, (1, 2), activation='relu', input_shape=(height, width, channels)))  # 필터 수 감소
cnn_model.add(MaxPooling2D(pool_size=(1, 2)))
cnn_model.add(Dropout(0.3))

cnn_model.add(Conv2D(32, (1, 2), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(1, 2)))
cnn_model.add(Dropout(0.3))

cnn_model.add(Flatten())
cnn_model.add(Dense(32, activation='relu'))  # 뉴런 수 감소
cnn_model.add(Dense(1))

# 모델 컴파일 (학습률 조정)
from keras.optimizers import Adam
optimizer = Adam(learning_rate=0.0001)  # 학습률 낮춤
cnn_model.compile(loss='mean_squared_error', optimizer=optimizer)

# EarlyStopping 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)

# 모델 학습
cnn_model.fit(X_train_cnn, y_train, epochs=200, batch_size=16, 
              validation_split=0.2,
              callbacks=[early_stopping])

# 모델 평가
cnn_score = cnn_model.evaluate(X_test_cnn, y_test)
print(f'Test Loss: {cnn_score}')


Epoch 1/200
167/167 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 104.6572 - val_loss: 93.7098
Epoch 2/200
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 88.0740 - val_loss: 71.1474
Epoch 3/200
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 60.9286 - val_loss: 41.9990
Epoch 4/200
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 36.9951 - val_loss: 27.3797
Epoch 5/200
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 29.7909 - val_loss: 23.3354
Epoch 6/200
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 25.5522 - val_loss: 21.4174
Epoch 7/200
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 25.5139 - val_loss: 19.6856
Epoch 8/200
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 23.8100 - val_loss: 17.8720
Epoch 9/200
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 20.3748 - val_loss: 16.3086
Epoch 10/200
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 26.5759 - val_loss: 14.8808
Epoch 11/200
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 19.5239 - val_loss: 14.1212
Epoch 12/200
167/1

### 결론적으로 처음에 간단하게 만든 모델이 가장 성능이 좋았다.
### CNN으로 전복데이터를 회귀하는 것은 부적합한 것 같다.